In [1]:
import pandas as pd
import requests
from tqdm.notebook import tqdm
import uuid
from sentence_transformers import SentenceTransformer
import pickle
from bs4 import BeautifulSoup
from rake_nltk import Rake

In [ ]:
data = pd.read_pickle('./data/top10k_docs_dict.pkl')

In [ ]:
topics = list(data.keys())

In [ ]:
record = data[topics[0]]
record[0]

In [ ]:
id = list(data.values())[0][0]

In [ ]:
idx = id[0:9]

In [ ]:
':'.join((idx, id))

In [ ]:
api_key = "83eaf18c-f42b-49e1-8d97-1062f00ac2eb"

In [ ]:
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')
bert_model.cuda()

In [ ]:
bert_model.max_seq_length = 1000

In [ ]:
bert_model.max_seq_length

In [ ]:
writefile = open('retrieved_encoded_docs.pkl', 'wb')
with open('./data/top10k_docs_dict.pkl', 'rb') as file:
    for i in range(298):
        print(i+1, 'of', 298)
        data = pickle.load(file)
        topic_id = list(data.keys())[0]
        record = list(data.values())[0]
        doc_list_encoded = []
        for i in tqdm(range(100)):
            id = record[i]
            index = id[0:9]
            doc_id = ':'.join((index, id))
            uid = uuid.uuid5(uuid.NAMESPACE_URL, doc_id)
            doc_index = 'cw09' if index == 'clueweb09' else 'cw12'
            url = 'https://www.chatnoir.eu/cache?apikey=' + api_key + '&uuid=' + str(uid) + '&index=' + doc_index +'&raw&plain'
            doc = requests.get(url).text
            doc = BeautifulSoup(doc, "lxml").text.replace('\n', ' ')
            doc_list_encoded.append(bert_model.encode(doc))
        add_dict = {topic_id: {
            'encoded_docs': doc_list_encoded
        }}
        pickle.dump(add_dict, writefile)

In [ ]:
data = pd.read_pickle('./data/top10k_docs_dict.pkl')
topics = list(data.keys())

with open('retrieved_docs_encoded.pkl', 'wb') as writefile:
  for i in range(len(topics)):
      print('Topic no:', i, '/298')
      record = data[topics[i]]
      topic_id = topics[i]
      doc_list_encoded = []
      doc_list = []
      for i in tqdm(range(100)):
          id = record[i]
          index = id[0:9]
          doc_id = ':'.join((index, id))
          uid = uuid.uuid5(uuid.NAMESPACE_URL, doc_id)
          doc_index = 'cw09' if index == 'clueweb09' else 'cw12'
          url = 'https://www.chatnoir.eu/cache?apikey=' + api_key + '&uuid=' + str(uid) + '&index=' + doc_index +'&raw&plain'
          doc = requests.get(url).text
          doc = BeautifulSoup(doc, "lxml").text.replace('\n', ' ')
          doc_list.append(doc)
          doc_list_encoded.append(bert_model.encode(doc))
      add_dict = {topic_id: {
          "docs": doc_list,
          "encoded_docs": doc_list_encoded
      }}
      pickle.dump(add_dict, writefile)

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
enc = np.array(data['encoded_docs'])
pca = PCA(n_components=2)
pca.fit(enc)

In [ ]:
enc_pca = pca.transform(enc)

In [ ]:
enc_pca

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(enc_pca[:, 0], enc_pca[:, 1])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(enc)

In [ ]:
data.keys()

In [2]:
res =  requests.get('https://www.chatnoir.eu/cache?apikey=83eaf18c-f42b-49e1-8d97-1062f00ac2eb&uuid=6f362e27-9991-5f93-a2d8-1b57f01e26a7&index=cw09&raw&plain')

In [3]:
txt = res.text

In [4]:
txt

'<!doctype html>\n<meta charset="utf-8">\n<title>A Letter written on Oct 30, 1937</title>\n<body>\n\n<h1>A Letter written on Oct 30, 1937</h1> \n<p> </p> \n<blockquote>  Department of Chemistry, Yenching University, Peiping, - October \n30, 1937.<br>\n\n<p> <b>Extracts from the Adolph Diary: 1937.</b> </p> \n<p> <em>January:</em> Ice skating on the Yenching lake at its prime. EHA, in \nthe 8th grade, North China American School, makes the hockey team. HMA, \nfreshman at Mount Holyoke College, amkes her second all-Holyoke athletic team \n(basketball). WHA Jr., sophomore at Yale University (Calhoun College), works in \nbiology and plays football and basketball on his college team. DMA (8 yrs.) \ntutors at home and specializes in dolls. - Through traffic opens on the trunk \nline railway from Canton to Hankow; great strides in national reconstruction.\n</p> \n<p> <em>February:</em> WHA serves on commission of five under the Council of \nPublic Health to prepare a code of nutrition standar

In [5]:
cleantext = BeautifulSoup(txt, "lxml").text.replace('\n', ' ')

In [6]:
cleantext

" A Letter written on Oct 30, 1937  A Letter written on Oct 30, 1937     Department of Chemistry, Yenching University, Peiping, - October  30, 1937.  Extracts from the Adolph Diary: 1937.   January: Ice skating on the Yenching lake at its prime. EHA, in  the 8th grade, North China American School, makes the hockey team. HMA,  freshman at Mount Holyoke College, amkes her second all-Holyoke athletic team  (basketball). WHA Jr., sophomore at Yale University (Calhoun College), works in  biology and plays football and basketball on his college team. DMA (8 yrs.)  tutors at home and specializes in dolls. - Through traffic opens on the trunk  line railway from Canton to Hankow; great strides in national reconstruction.   February: WHA serves on commission of five under the Council of  Public Health to prepare a code of nutrition standards for China. Cereals now  supply 85 percent of the food intake; we urge that this be replaced in part by  more green vegetables and legumes. Milk is not avail

In [ ]:
r = Rake()
r.extract_keywords_from_text(cleantext)
len(r.get_ranked_phrases()[0])

In [ ]:
doc_list_encoded

In [ ]:
f = open('retrieved_docs_encoded.pkl', 'rb')
pickle.load(f)

In [ ]:
id = uuid.uuid5(uuid.NAMESPACE_URL, "clueweb09:clueweb09-en0008-56-34223")
id

In [ ]:
url = "https://www.chatnoir.eu/api/v1/_search"
data = {
    "apikey": "83eaf18c-f42b-49e1-8d97-1062f00ac2eb",
    "query": "clueweb09-en0008-56-34223",
    "index": ["cw09", "cw12"],
    "pretty": True
}

In [ ]:
res = requests.post(url, data = data, timeout=2.50)

In [ ]:
res = list(res)

In [ ]:
res.json()

In [ ]:
res[1]

In [ ]:
res =  requests.get('https://www.chatnoir.eu/cache?apikey=83eaf18c-f42b-49e1-8d97-1062f00ac2eb&uuid=6f362e27-9991-5f93-a2d8-1b57f01e26a7&index=cw09&raw&plain')

In [ ]:
res.text

In [ ]:
f = open('retrieved_docs_encoded.pkl', 'rb')

In [ ]:
data = pickle.load(f)
data.keys()

In [ ]:
#from sklearn.decomposition import PCA
#from sklearn.cluster import KMeans
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
data = list(data.values())[0]

In [ ]:
enc = np.array(data['encoded_docs'])[:50]

In [ ]:
pca = PCA(n_components=2)
pca.fit(enc)

In [ ]:
enc_pca = pca.transform(data['encoded_docs'])

In [ ]:
plt.scatter(enc_pca[:, 0], enc_pca[:, 1])

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(enc_pca)

In [ ]:
labels = kmeans.labels_

In [ ]:
filtered_enc_pca_0 = enc_pca[labels == 0]

In [ ]:
plt.scatter(filtered_enc_pca_0[:, 0], filtered_enc_pca_0[:, 1])

In [ ]:
filtered_enc = enc[labels == 0]

In [ ]:
similarity = sklearn.metrics.pairwise.cosine_similarity(filtered_enc)
sklearn.metrics.pairwise.cosine_similarity(filtered_enc)

In [ ]:
np.mean(np.mean(similarity))

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
enc = enc[:50]

In [ ]:
clustering = DBSCAN().fit(enc)

In [ ]:
clustering.labels_

In [ ]:
enc_0 = enc[clustering.labels_ == 0]
enc_1 = enc[clustering.labels_ == -1]

In [ ]:
pca = PCA(n_components = 2)

In [ ]:
pca.fit(enc)

In [ ]:
enc_0_pca = pca.transform(enc_0)

In [ ]:
enc_1_pca = pca.transform(enc_1)

In [ ]:
#plt.scatter(enc_0_pca[:, 0], enc_0_pca[:, 1])
plt.scatter(enc_1_pca[:, 0], enc_1_pca[:, 1])

In [ ]:
import umap

In [ ]:
reducer = umap.UMAP()

In [ ]:
scaled_data = StandardScaler().fit_transform(enc)

In [ ]:
embedding = reducer.fit_transform(scaled_data)
embedding.shape

In [ ]:
plt.scatter(
    embedding[:, 0],
    embedding[:, 1])

In [ ]:
r = Rake()

In [ ]:
f = open('retrieved_docs_encoded.pkl', 'rb')

with open('retrieved_docs_keywords.pkl', 'wb') as writefile:
  for i in range(298):
      print('Topic no:', i, '/298')
      data = pickle.load(f)
      topic_id = list(data.keys())[0]
      record = list(data.values())[0]
      doc_list = record['docs']
      kw_list = []
      summary_list = []
      for j in tqdm(range(100)):
          doc = doc_list[j]
          r.extract_keywords_from_text(doc)
          klist = r.get_ranked_phrases()
          summary = '. '.join(klist[:100])
          kw_list.append(klist)
          summary_list.append(summary)
      add_dict = {topic_id: {
          "docs": doc_list,
          "keywords": kw_list,
          "summary": summary_list
      }}
      pickle.dump(add_dict, writefile)

In [ ]:
f = open('retrieved_docs_encoded.pkl', 'rb')

In [ ]:
data = pickle.load(f)
data